# Cancer Mortality Dataset 1980 - 2014

In [167]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from highcharts import Highmap

import json

In [168]:
# Read in Mortality csv as dataframe
df = pd.read_csv("mort.csv", na_values=[' '])

In [169]:
# Only observe Neoplasm(Cancer) related mortality
cancer = df.loc[df['Category'] == 'Neoplasms']
print list(cancer.columns.values)

['Location', 'FIPS', 'Category', 'Mortality Rate, 1980*', 'Mortality Rate, 1980* (Min)', 'Mortality Rate, 1980* (Max)', 'Mortality Rate, 1985*', 'Mortality Rate, 1985* (Min)', 'Mortality Rate, 1985* (Max)', 'Mortality Rate, 1990*', 'Mortality Rate, 1990* (Min)', 'Mortality Rate, 1990* (Max)', 'Mortality Rate, 1995*', 'Mortality Rate, 1995* (Min)', 'Mortality Rate, 1995* (Max)', 'Mortality Rate, 2000*', 'Mortality Rate, 2000* (Min)', 'Mortality Rate, 2000* (Max)', 'Mortality Rate, 2005*', 'Mortality Rate, 2005* (Min)', 'Mortality Rate, 2005* (Max)', 'Mortality Rate, 2010*', 'Mortality Rate, 2010* (Min)', 'Mortality Rate, 2010* (Max)', 'Mortality Rate, 2014*', 'Mortality Rate, 2014* (Min)', 'Mortality Rate, 2014* (Max)', '% Change in Mortality Rate, 1980-2014', '% Change in Mortality Rate, 1980-2014 (Min)', '% Change in Mortality Rate, 1980-2014 (Max)']


In [170]:
# Convert the float FIPS value into a padded string value usable by highmap lib
def converter(x):
    return x['FIPS'].astype(int).astype(str).zfill(5)

visualize = cancer[['FIPS','% Change in Mortality Rate, 1980-2014']]

# Remove NaN FIPS values
visualize = visualize[np.isfinite(visualize['FIPS'])]
states = visualize[visualize['FIPS'] <= 56]
counties = visualize[visualize['FIPS'] > 56]


In [171]:
# Counties - Set up variance and step sizes for future color selection
C_maximum = counties['% Change in Mortality Rate, 1980-2014'].max()
C_minimum = counties['% Change in Mortality Rate, 1980-2014'].min()

C_variance = C_maximum - C_minimum

C_step = C_variance/6

In [172]:
# Counties - Trim down to usable list of dicts
counties.loc[:,'FIPS'] = counties.apply(converter, axis=1)

counties.columns = ['code', 'value']
counties = counties.T.to_dict().values()

In [177]:
C = Highmap(width = 800, height = 700)

# Read in county options
with open('county_map_options.json', 'r') as f:
     C_options = json.load(f)


C.set_dict_options(C_options) # set options
C.add_data_set(counties, 'map', 'Change in Mortality', joinBy=['fips', 'code'])

C.set_map_source('http://code.highcharts.com/mapdata/countries/us/us-all-all.js', jsonp_map=False)

C
